# Geocoding and Network Analysis Demo

## What is the Network Analysis?

The ArcGIS Network Analyst allows you to solve common `network` problems



### Examples:

- finding the best route across a city
- finding the closest emergency vehicle or facility, 
- identifying a service area around a location
- servicing a set of orders with a fleet of vehicles 
- choosing the best facilities to open or close.

# Constructing drive time based service areas
This sample shows how the `network` module of the ArcGIS API for Python can be used to construct service areas. In this sample, we generate service areas for two of the fire stations in central Tokyo, Japan. We later observe how the service area varies by time of day for a fire station in the city of Los Angeles.

##### Connect to the GIS

In [3]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS

my_gis = GIS(profile='agol_profile')

### Create a Network Layer

In [4]:
from arcgis.network import ServiceAreaLayer

In [5]:
service_area_url = my_gis.properties.helperServices.serviceArea.url
service_area_url

'https://routedev.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [6]:
sa_layer = ServiceAreaLayer(service_area_url, gis=my_gis)

## Calculate service area from the Resort

In [7]:
from arcgis.geocoding import geocode
ps_geocode = geocode('Hilton, Berlin, DE', as_featureset=True)
sdf = ps_geocode.sdf
sdf.head()

,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Addr_type,BldgName,BldgType,Block,City,...,URL,UnitName,UnitType,X,Xmax,Xmin,Y,Ymax,Ymin,Zone
0,,30,,,,POI,,,,Berlin,...,http://www3.hilton.com/en/hotels/berlin/hilton...,,,13.39297,13.39782,13.38782,52.51228,52.51717,52.50717,
1,,69,,,,POI,,,,Berlin,...,http://hamptoninn3.hilton.com/en/hotels/berlin...,,,13.41880,13.42363,13.41363,52.52443,52.52949,52.51949,


# Display the Hilton Hotels in Berlin

In [8]:
map1 = my_gis.map()
map1.zoomLevel = 10
map1.extent = sdf.spatial.full_extent
map1

MapView(layout=Layout(height='400px', width='100%'))

In [24]:
ps_geocode.sdf.spatial.plot(map_widget=map1)

True

## Compute the service area


In [10]:
travel_modes = sa_layer.retrieve_travel_modes()

for t in travel_modes['supportedTravelModes']:
    print(t['name'])

Driving Time
Driving Distance
Trucking Time
Trucking Distance
Walking Time
Walking Distance
Rural Driving Time
Rural Driving Distance


In [11]:
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

In [12]:
sdf = ps_geocode.sdf
s = sdf.spatial

In [13]:
s.to_feature_collection()

<FeatureCollection>

In [ ]:
from arcgis.features import analysis
drive_time_job = analysis.create_drive_time_areas(s.to_feature_collection(), 
                                       break_values=[5, 10, 15],
                                       travel_mode=truck_mode,
                                       future=True)
drive_time_job

## Checking Job Status

In [20]:
drive_time_job.status

'esriJobSucceeded'

## Get the Job Result

In [22]:
fc = drive_time_job.result()
fc

<FeatureCollection>

### Visualize the service area on the map

Visualize the results on the map widget.  


In [23]:
map1.add_layer(fc)

Click the drive time areas to explore their attributes. Because the content of the pop-ups may include HTML source code, it is also possible to have the pop-up windows include other resources such as tables and images.

### Driving directions

In [28]:
from arcgis.network import RouteLayer

In [29]:
route_service_url = my_gis.properties.helperServices.route.url
route_service_url

'https://routedev.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World'

In [30]:
route_service = RouteLayer(route_service_url, gis=my_gis)
route_service

<RouteLayer url:"https://routedev.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World">

In [55]:
stops = [ 'Zufahrt zum Flughafen Tegel, 13405 Berlin, Germany', 'Mohrenstraße 30, 10117 Berlin, Germany']
from arcgis.geocoding import geocode, batch_geocode
stops_geocoded = batch_geocode(stops)

#### data wrangling

In [56]:
stops_geocoded = [item['location'] for item in stops_geocoded]
stops_geocoded

[{'x': 13.309110351000072, 'y': 52.55763923600006},
 {'x': 13.392801147000057, 'y': 52.51226653600003}]

In [57]:
stops_geocoded2 = '{},{};{},{}'.format(stops_geocoded[0]['x'],stops_geocoded[0]['y'],
                                       stops_geocoded[1]['x'],stops_geocoded[1]['y'])
stops_geocoded2

'13.309110351000072,52.55763923600006;13.392801147000057,52.51226653600003'

### Driving directions

In [58]:
result = route_service.solve(stops_geocoded2, 
                             return_routes=True,
                             return_stops=True, 
                             return_directions=True,
                             return_barriers=False, 
                             return_polygon_barriers=False,
                             return_polyline_barriers=False)

In [65]:
ext = {'spatialReference': {'latestWkid': 4326},
       'xmin': 13.309110351000072,
       'ymin': 52.55763923600006,
       'xmax': 13.392801147000057,
       'ymax': 52.51226653600003}

In [69]:
map2 = my_gis.map('Berlin, Germany', zoomlevel=12)

map2

MapView(layout=Layout(height='400px', width='100%'), zoom=12.0)

In [61]:
from arcgis.features import Feature, FeatureSet
features = result['routes']['features']
routes = FeatureSet(features)
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)

In [62]:
len(result['stops'])

4

In [63]:
stop_features = result['stops']['features']
stop_fset = FeatureSet(stop_features)

In [70]:
ln_symbol = { "type": "simple-line", "width": 2, "color": [255, 0, 0, 1] }
symbol = {
    "type" : "simple-marker",
    "style" : "square",
    "outline" : {
        "style": "dash-dot"
    },
    "color": [255, 211, 127, 1]
}
map2.draw(routes, symbol=ln_symbol)
map2.draw(stop_fset, symbol=symbol)

# Visualizing the Instructions

In [71]:
import pandas as pd
text = [att['attributes'] for att in result['directions'][0]['features']]
df = pd.DataFrame(text)[['text', 'length','time']]
df

,text,length,time
0,Start at Location 1,0.000000,0.000000
1,Go southwest on A111 (Tunnel Flughafen Tegel) ...,1.196488,1.769828
2,Take the ramp on the right at exit 12 to Hecke...,0.188789,0.455830
3,Continue forward on Kurt-Schumacher-Damm,0.358532,0.940048
4,"At the roundabout, take the second exit onto T...",0.599046,1.425649
5,Turn left on Osnabrücker Straße,0.388085,1.072980
6,Continue forward on Kaiserin-Augusta-Allee,0.917693,2.281182
7,Continue forward on Alt-Moabit,0.102219,0.483672
8,Turn right on Gotzkowskystraße,0.062420,0.303635
9,Bear left on Levetzowstraße,0.408630,1.129607
